# 1. 환경 설치

In [ ]:
# conda 환경 생성, conda create -n {환경이름} python=3.12.8
# pip install ipykernel
# pip install pillow exifread geopy datetime
from PIL import Image
import exifread
from geopy.point import Point
from geopy.geocoders import Nominatim
import os

In [ ]:
# pip install tensorflow
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image

In [ ]:
# pip install opencv-python pytesseract
# conda install -c conda-forge tesseract

import cv2
import pytesseract

# 아래 경로는 터미널에서 where tesseract 입력 시 출력되는 경로를 넣어준다.
pytesseract.pytesseract.tesseract_cmd = (
    r"C:\Users\Admin\.conda\envs\flashback_env\Library\bin\tesseract.exe"
)

In [2]:
def get_exif_data(image_path):
    geolocator = Nominatim(user_agent="South Korea")
    
    with open(image_path, 'rb') as f:
        tags = exifread.process_file(f)
    # GPS 정보 추출
    latitude = tags.get('GPS GPSLatitude')
    longitude = tags.get('GPS GPSLongitude')
    timestamp = tags.get('EXIF DateTimeOriginal')
    
    
    if latitude and longitude:
        latitude_sec = float(latitude.values[2].num) / float(latitude.values[2].den)
        longitude_sec = float(longitude.values[2].num) / float(longitude.values[2].den)

        latitude_dms = f"{latitude.values[0]} {latitude.values[1]}m {latitude_sec}s N"
        longitude_dms = f"{longitude.values[0]} {longitude.values[1]}m {longitude_sec}s E"
        
        # location = "beep"
        point = Point(latitude_dms + " " + longitude_dms)
        
        # location = rg.search((point.latitude, point.longitude))
        location = geolocator.reverse((point.latitude, point.longitude))
        
        # 구글, 네이버, 카카오 등 시험해볼것
    else:
        location = None

    return {
        "location": location[:-1] if location else "위치 정보 없음",
        "timestamp": timestamp.printable if timestamp else "시간 정보 없음"
    }

In [3]:
img_folder = "img"
paths = [
    os.path.join(img_folder, file)
    for file in os.listdir(img_folder)
    if os.path.isfile(os.path.join(img_folder, file))
]

for path in paths:
  print(get_exif_data(path))

{'location': '위치 정보 없음', 'timestamp': '시간 정보 없음'}


In [5]:
# ResNet50 모델 로드
model = ResNet50(weights='imagenet')

def predict_location(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    preds = model.predict(img_array)
    decoded_preds = decode_predictions(preds, top=3)[0]  # 상위 3개 후보 반환
    
    return [(label, prob) for (_, label, prob) in decoded_preds]

In [6]:
for path in paths:
  print(predict_location(path))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 869ms/step
[('digital_watch', np.float32(0.3794299)), ('digital_clock', np.float32(0.22164412)), ('stopwatch', np.float32(0.11135533))]


In [17]:
def extract_text(img_path):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    text = pytesseract.image_to_string(gray, lang="eng+kor")  # 영어+한글 OCR
    # text = pytesseract.image_to_string(gray, lang="eng")  # 영어 OCR
    # text = pytesseract.image_to_string(gray, lang="kor")  # 한글 OCR
    return text

In [18]:
ocr_img_folder = "ocr_img"
paths = [
    os.path.join(ocr_img_folder, file)
    for file in os.listdir(ocr_img_folder)
    if os.path.isfile(os.path.join(ocr_img_folder, file))
]

for path in paths:

  print(f"[{path}]")

  print(extract_text(path))

[ocr_img\test.jpg]
tesseractS
이용한
OCR 테스트

[ocr_img\test2.png]
This is a lot of 12 point text to test the
ocr code and see if it works on all types
of file format.

The quick brown dog jumped over the
lazy fox. The quick brown dog jumped
over the lazy fox. The quick brown dog
jumped over the lazy fox. The quick
brown dog jumped over the lazy fox.

[ocr_img\test3.png]
때 비

실험

심리

WWsers>tesseract D:/Data/ocr/sample.jpa stdout -| kor_Istm_best —-psm 4 --060 1

-

회 ou 더
페 훈 힌 호 호
mw oS 힐 도
2m 좋 ow
Ss ° Was
후 버 Ul ar 페 전
sea 5 a y w
aS 2 i &
a a wor &
해조 물 떠 Ss
zm mH oO ge
fw = iw or
. = 2 a wr
Ss, ™ emt
w sobs w
em Fox oy Wl
aie 증오
때 Gy Fs
7 4 go 25
oT i 을 뽀
Rr Se <1 hOB ie
Wy oe
=a iW a =
Noa 해 에 =
띠 호경 딱히 &
이 응 2a i)
2 8 aT -
My VB So
더 a Sp
axzghsn 8
ze Ps Fw =
ao 이방 호
asp os
이 ym ROB
wo <u By 그 바고
1 ier 5 cm 헤
Sy! Smo
어버이 = Sys
KROL, ties
@ 112 HP HERO
wR? Bi Oy
RO Sah Oe

은 금 버 올 느 응

구아바 Lolz Barer
SEW 5 Finan PSA
ae Polaige “lita
Stes y BIW ay,
Sun SAMS MO
ssh Yow2 x: eae
ow ecaS